In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("file.csv", sep=';', encoding='latin1', na_values='idk', decimal=',')

In [3]:
df.head()

,Argus Drug Code,Product Name,ICRC Name,TradeName,Category (0-5),Category (new),TA,Responsible GSL,TA Coordinator,DLP,...,Planning Outsource,outsourced to,score,Time,Complexity,# of Topics,Complexity New,Pages,Indications,Products
0,BAY39-6251,AZELAIC ACID,0,Skinoren,2,B,GM/DI,Bianca Gagliotti,Asad Kazi,02.01.2017,...,yes,TCS,1,93.0,III,10,III,131.0,3.0,1.0
1,BAY1841788,Darolutamide (previous: ODM-201 (AR Antagonist)),ODM-201,NaN,1,A,ONC Non-TKI,Ping Ji,Susanne Kippes,04.01.2017,...,yes,ICRC-Weyer,1,124.5,III,6,IV,349.0,NaN,1.0
2,BAYA2502,NIFURTIMOX,0,Lampit,2,B,PC/Pul,Yuhong Wang,Kathrin Mastrangelo,07.01.2017,...,yes,TCS,1,98.0,III,2,V,147.0,1.0,1.0
3,BAYA2502,NIFURTIMOX,0,Lampit,2,B,PC/Pul,Yuhong Wang,Kathrin Mastrangelo,07.01.2017,...,yes,TCS,1,106.0,III,8,III,56.0,2.0,1.0
4,BAY1217389,"MPS1 INHIBIOTR ""FU"" (imidazopyr)",0,NaN,1,A,ONC Non-TKI,Cinara McCarthy,Susanne Kippes,20.01.2017,...,yes,TCS,1,54.0,III,1,V,272.0,1.0,1.0


In [4]:
y = df['Time']

In [5]:
X = df.loc[:, df.columns!='Time']


In [6]:
# Checking Pearson
df[df.columns[1:]].corr()['Time']

Category (0-5)                                          0.278319
Due Date \n(if no exact date mid of quarter planned)         NaN
score                                                        NaN
Time                                                    1.000000
# of Topics                                             0.434516
Pages                                                   0.012926
Indications                                            -0.046333
Products                                                0.174516
Name: Time, dtype: float64

In [7]:
df = df[['# of Topics', 'Products','TA']]

In [8]:
# Data cleaning
# Convert outsourced to to numeric
from sklearn import preprocessing as pre
# le = pre.LabelEncoder()
# le.fit(df['outsourced to'])
# df['outsourced to'] = le.transform(df['outsourced to'])

In [9]:
# Same for TA
le = pre.LabelEncoder()
le.fit(df['TA'])
df['TA'] = le.transform(df['TA'])

In [10]:
# pages: impute nans with mean
# from sklearn.preprocessing import Imputer
# imp = Imputer()
# imp.fit(df['Pages'].reshape(-1, 1))
# new = imp.transform(df['Pages'].reshape(-1,1))
# df['Pages'] = new

In [11]:
# Topics is fine
# Indications: Replace idk & NaNs with 1
df.Indications = df.Indications.fillna(value=1, )

AttributeError: 'DataFrame' object has no attribute 'Indications'

In [12]:
df.Products = df.Products.fillna(value=1)


In [13]:
X = df.copy()

In [14]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

C:\Users\hagen\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
from sklearn.ensemble import RandomForestRegressor
rf  = RandomForestRegressor()
rf.fit(X, y)
print('Features by their score')
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), ['Topics', 'Products', 'TA']), reverse=True))

Features by their score
[(0.68300000000000005, 'Topics'), (0.18379999999999999, 'Products'), (0.1331, 'TA')]


In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [17]:
# Try linear regression
from sklearn.linear_model import LinearRegression as lr
my_lr = lr()
my_lr.fit(X_train_std, y_train)
my_lr.predict(X_test_std)
my_lr.score(X=X_test_std, y=y_test)
#my_lr.coef_

0.50900774630334555

In [18]:
from sklearn.svm import SVR 
my_svr  = SVR(C=1, epsilon=0.2)
my_svr.fit(X_train_std, y_train)
my_lr.score(X=X_test_std, y=y_test)

0.50900774630334555

In [19]:
#df = df[['outsourced to', '# of Topics', 'Indications', 'Products', 'TA']]


In [20]:
# from sklearn.neural_network import MLPRegressor
# my_net = MLPRegressor(hidden_layer_sizes=(3,3,1), activation='relu', alpha=0.0001, max_iter=1000)
# my_net.fit(X_train_std, y_train)
# my_net.score(X=X_test_std, y=y_test)

In [21]:
# from sklearn.model_selection import GridSearchCV
# alpha = 10.0 ** -np.arange(1,7)
# iterations = [500, 1000, 5000]
# params = {
#             'hidden_layer_sizes' : ((1,1,1), (2,2,2), (4,5,1), (3,3,1)),
#             'alpha' : alpha, 
#             'activation' : ('relu', 'logistic'), 
#             'solver' : ('adam', 'sgd'),
#             'max_iter': iterations
#          }
# my_clf = GridSearchCV(my_net, params)
# my_clf.fit(X_train_std, y_train)
# my_clf.score(X=X_test_std, y=y_test)

In [51]:
my_clf.best_estimator_

MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 3, 1), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)